In [5]:
import cv2
import time
import psutil

# Load Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Start webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Benchmark variables
start_session = time.time()
max_duration = 10  # seconds
fps_list = []
cpu_list = []
total_faces_detected = 0
frame_counter = 0
min_fps = float('inf')
max_fps = 0

print(f"[INFO] Running Haar Cascade benchmark for {max_duration} seconds...")

while True:
    elapsed_time = time.time() - start_session
    if elapsed_time > max_duration:
        break

    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read frame.")
        break

    frame_counter += 1
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    t0 = time.time()
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    t1 = time.time()

    frame_fps = 1 / (t1 - t0)
    fps_list.append(frame_fps)
    total_faces_detected += len(faces)
    min_fps = min(min_fps, frame_fps)
    max_fps = max(max_fps, frame_fps)

    # Get CPU usage
    cpu_usage = psutil.cpu_percent(interval=0)
    cpu_list.append(cpu_usage)

    # Draw rectangles
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Display live stats
    avg_fps = sum(fps_list) / len(fps_list)
    avg_cpu = sum(cpu_list) / len(cpu_list)

    stats = [
        f"Faces: {len(faces)}",
        f"FPS (curr): {frame_fps:.2f}",
        f"FPS (avg): {avg_fps:.2f}",
        f"FPS (min): {min_fps:.2f}",
        f"FPS (max): {max_fps:.2f}",
        f"CPU: {cpu_usage:.1f}%",
        f"Frames: {frame_counter}",
        f"Total Faces: {total_faces_detected}",
        f"Elapsed: {elapsed_time:.1f}s"
    ]

    for i, stat in enumerate(stats):
        cv2.putText(frame, stat, (10, 25 + i * 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # Show the frame
    cv2.imshow("Haar Cascade Live Benchmark (10 sec)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# Final benchmark summary
print("\n[INFO] Benchmark Session Summary:")
print(f" - Total frames processed: {frame_counter}")
print(f" - Total faces detected: {total_faces_detected}")
print(f" - Min FPS: {min_fps:.2f}")
print(f" - Max FPS: {max_fps:.2f}")
print(f" - Avg FPS: {sum(fps_list)/len(fps_list):.2f}")
print(f" - Avg CPU Usage: {sum(cpu_list)/len(cpu_list):.2f}%")
print(f" - Elapsed time: {elapsed_time:.2f} seconds")


[INFO] Running Haar Cascade benchmark for 10 seconds...

[INFO] Benchmark Session Summary:
 - Total frames processed: 154
 - Total faces detected: 78
 - Min FPS: 8.47
 - Max FPS: 45.64
 - Avg FPS: 26.83
 - Avg CPU Usage: 54.42%
 - Elapsed time: 10.03 seconds


In [7]:
import cv2
import time
import psutil
from facenet_pytorch import MTCNN
import torch

# Check if GPU is available (optional)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"[INFO] Using device: {device}")

# Initialize MTCNN face detector
mtcnn = MTCNN(keep_all=True, device=device)

# Start webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Benchmark variables
start_session = time.time()
max_duration = 10  # seconds
fps_list = []
cpu_list = []
total_faces_detected = 0
frame_counter = 0
min_fps = float('inf')
max_fps = 0

print(f"[INFO] Running MTCNN benchmark for {max_duration} seconds...")

while True:
    elapsed_time = time.time() - start_session
    if elapsed_time > max_duration:
        break

    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read frame.")
        break

    frame_counter += 1
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    t0 = time.time()
    boxes, _ = mtcnn.detect(rgb_frame)
    t1 = time.time()

    frame_fps = 1 / (t1 - t0)
    fps_list.append(frame_fps)
    cpu_usage = psutil.cpu_percent(interval=0)
    cpu_list.append(cpu_usage)

    # Count faces and draw boxes
    num_faces = 0
    if boxes is not None:
        num_faces = len(boxes)
        for box in boxes:
            x1, y1, x2, y2 = [int(coord) for coord in box]
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 128, 255), 2)

    total_faces_detected += num_faces
    min_fps = min(min_fps, frame_fps)
    max_fps = max(max_fps, frame_fps)
    avg_fps = sum(fps_list) / len(fps_list)
    avg_cpu = sum(cpu_list) / len(cpu_list)

    # Display stats
    stats = [
        f"Faces: {num_faces}",
        f"FPS (curr): {frame_fps:.2f}",
        f"FPS (avg): {avg_fps:.2f}",
        f"FPS (min): {min_fps:.2f}",
        f"FPS (max): {max_fps:.2f}",
        f"CPU: {cpu_usage:.1f}%",
        f"Frames: {frame_counter}",
        f"Total Faces: {total_faces_detected}",
        f"Elapsed: {elapsed_time:.1f}s"
    ]

    for i, stat in enumerate(stats):
        cv2.putText(frame, stat, (10, 25 + i * 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    cv2.imshow("MTCNN Live Benchmark (10 sec)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# Final summary
print("\n[INFO] MTCNN Benchmark Summary:")
print(f" - Total frames processed: {frame_counter}")
print(f" - Total faces detected: {total_faces_detected}")
print(f" - Min FPS: {min_fps:.2f}")
print(f" - Max FPS: {max_fps:.2f}")
print(f" - Avg FPS: {avg_fps:.2f}")
print(f" - Avg CPU Usage: {avg_cpu:.2f}%")
print(f" - Elapsed time: {elapsed_time:.2f} seconds")


/home/maksimgorki/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maksimgorki/.local/lib/python3.12/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


[INFO] Using device: cpu
[INFO] Running MTCNN benchmark for 10 seconds...

[INFO] MTCNN Benchmark Summary:
 - Total frames processed: 141
 - Total faces detected: 142
 - Min FPS: 12.06
 - Max FPS: 22.03
 - Avg FPS: 16.83
 - Avg CPU Usage: 60.64%
 - Elapsed time: 10.03 seconds


In [8]:
import cv2
import time
import psutil
import mediapipe as mp

# Initialize MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
face_detection = mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5)

# Open webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Benchmark variables
start_session = time.time()
max_duration = 10  # seconds
fps_list = []
cpu_list = []
total_faces_detected = 0
frame_counter = 0
min_fps = float('inf')
max_fps = 0

print(f"[INFO] Running MediaPipe benchmark for {max_duration} seconds...")

while True:
    elapsed_time = time.time() - start_session
    if elapsed_time > max_duration:
        break

    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read frame.")
        break

    frame_counter += 1
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    t0 = time.time()
    results = face_detection.process(rgb_frame)
    t1 = time.time()

    frame_fps = 1 / (t1 - t0)
    fps_list.append(frame_fps)
    cpu_usage = psutil.cpu_percent(interval=0)
    cpu_list.append(cpu_usage)

    num_faces = 0
    if results.detections:
        num_faces = len(results.detections)
        for detection in results.detections:
            mp_drawing.draw_detection(frame, detection)

    total_faces_detected += num_faces
    min_fps = min(min_fps, frame_fps)
    max_fps = max(max_fps, frame_fps)
    avg_fps = sum(fps_list) / len(fps_list)
    avg_cpu = sum(cpu_list) / len(cpu_list)

    # Display stats
    stats = [
        f"Faces: {num_faces}",
        f"FPS (curr): {frame_fps:.2f}",
        f"FPS (avg): {avg_fps:.2f}",
        f"FPS (min): {min_fps:.2f}",
        f"FPS (max): {max_fps:.2f}",
        f"CPU: {cpu_usage:.1f}%",
        f"Frames: {frame_counter}",
        f"Total Faces: {total_faces_detected}",
        f"Elapsed: {elapsed_time:.1f}s"
    ]

    for i, stat in enumerate(stats):
        cv2.putText(frame, stat, (10, 25 + i * 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    cv2.imshow("MediaPipe Face Detection Benchmark (10 sec)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# Final summary
print("\n[INFO] MediaPipe Benchmark Summary:")
print(f" - Total frames processed: {frame_counter}")
print(f" - Total faces detected: {total_faces_detected}")
print(f" - Min FPS: {min_fps:.2f}")
print(f" - Max FPS: {max_fps:.2f}")
print(f" - Avg FPS: {avg_fps:.2f}")
print(f" - Avg CPU Usage: {avg_cpu:.2f}%")
print(f" - Elapsed time: {elapsed_time:.2f} seconds")


2025-03-24 01:26:14.204691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742768774.217318   42461 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742768774.221164   42461 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742768774.231415   42461 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742768774.231428   42461 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742768774.231430   42461 computation_placer.cc:177] computation placer alr

[INFO] Running MediaPipe benchmark for 10 seconds...

[INFO] MediaPipe Benchmark Summary:
 - Total frames processed: 154
 - Total faces detected: 154
 - Min FPS: 7.26
 - Max FPS: 339.62
 - Avg FPS: 109.62
 - Avg CPU Usage: 21.45%
 - Elapsed time: 10.01 seconds


In [9]:
import cv2
import dlib
import time
import psutil

# Initialize Dlib's face detector (HOG-based)
detector = dlib.get_frontal_face_detector()

# Open webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Benchmark variables
start_session = time.time()
max_duration = 10  # seconds
fps_list = []
cpu_list = []
total_faces_detected = 0
frame_counter = 0
min_fps = float('inf')
max_fps = 0

print(f"[INFO] Running Dlib benchmark for {max_duration} seconds...")

while True:
    elapsed_time = time.time() - start_session
    if elapsed_time > max_duration:
        break

    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read frame.")
        break

    frame_counter += 1
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    t0 = time.time()
    faces = detector(gray)
    t1 = time.time()

    frame_fps = 1 / (t1 - t0)
    fps_list.append(frame_fps)
    cpu_usage = psutil.cpu_percent(interval=0)
    cpu_list.append(cpu_usage)

    num_faces = len(faces)
    total_faces_detected += num_faces
    min_fps = min(min_fps, frame_fps)
    max_fps = max(max_fps, frame_fps)
    avg_fps = sum(fps_list) / len(fps_list)
    avg_cpu = sum(cpu_list) / len(cpu_list)

    # Draw rectangles around faces
    for face in faces:
        x1, y1, x2, y2 = face.left(), face.top(), face.right(), face.bottom()
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

    # Display stats
    stats = [
        f"Faces: {num_faces}",
        f"FPS (curr): {frame_fps:.2f}",
        f"FPS (avg): {avg_fps:.2f}",
        f"FPS (min): {min_fps:.2f}",
        f"FPS (max): {max_fps:.2f}",
        f"CPU: {cpu_usage:.1f}%",
        f"Frames: {frame_counter}",
        f"Total Faces: {total_faces_detected}",
        f"Elapsed: {elapsed_time:.1f}s"
    ]

    for i, stat in enumerate(stats):
        cv2.putText(frame, stat, (10, 25 + i * 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

    cv2.imshow("Dlib Face Detection Benchmark (10 sec)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# Final benchmark summary
print("\n[INFO] Dlib Benchmark Summary:")
print(f" - Total frames processed: {frame_counter}")
print(f" - Total faces detected: {total_faces_detected}")
print(f" - Min FPS: {min_fps:.2f}")
print(f" - Max FPS: {max_fps:.2f}")
print(f" - Avg FPS: {avg_fps:.2f}")
print(f" - Avg CPU Usage: {avg_cpu:.2f}%")
print(f" - Elapsed time: {elapsed_time:.2f} seconds")


[INFO] Running Dlib benchmark for 10 seconds...

[INFO] Dlib Benchmark Summary:
 - Total frames processed: 150
 - Total faces detected: 96
 - Min FPS: 14.23
 - Max FPS: 42.48
 - Avg FPS: 34.78
 - Avg CPU Usage: 16.49%
 - Elapsed time: 10.01 seconds


In [11]:
import cv2
import face_recognition
import time
import psutil

# Start webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Benchmark variables
start_session = time.time()
max_duration = 10  # seconds
fps_list = []
cpu_list = []
total_faces_detected = 0
frame_counter = 0
min_fps = float('inf')
max_fps = 0

print(f"[INFO] Running face_recognition benchmark for {max_duration} seconds...")

while True:
    elapsed_time = time.time() - start_session
    if elapsed_time > max_duration:
        break

    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to read frame.")
        break

    frame_counter += 1
    rgb_frame = frame[:, :, ::-1]  # Convert BGR to RGB

    t0 = time.time()
    face_locations = face_recognition.face_locations(rgb_frame)
    t1 = time.time()

    frame_fps = 1 / (t1 - t0)
    fps_list.append(frame_fps)
    cpu_usage = psutil.cpu_percent(interval=0)
    cpu_list.append(cpu_usage)

    num_faces = len(face_locations)
    total_faces_detected += num_faces
    min_fps = min(min_fps, frame_fps)
    max_fps = max(max_fps, frame_fps)
    avg_fps = sum(fps_list) / len(fps_list)
    avg_cpu = sum(cpu_list) / len(cpu_list)

    # Draw boxes around faces
    for top, right, bottom, left in face_locations:
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

    # Display stats
    stats = [
        f"Faces: {num_faces}",
        f"FPS (curr): {frame_fps:.2f}",
        f"FPS (avg): {avg_fps:.2f}",
        f"FPS (min): {min_fps:.2f}",
        f"FPS (max): {max_fps:.2f}",
        f"CPU: {cpu_usage:.1f}%",
        f"Frames: {frame_counter}",
        f"Total Faces: {total_faces_detected}",
        f"Elapsed: {elapsed_time:.1f}s"
    ]

    for i, stat in enumerate(stats):
        cv2.putText(frame, stat, (10, 25 + i * 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

    cv2.imshow("face_recognition Benchmark (10 sec)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()

# Final benchmark summary
print("\n[INFO] face_recognition Benchmark Summary:")
print(f" - Total frames processed: {frame_counter}")
print(f" - Total faces detected: {total_faces_detected}")
print(f" - Min FPS: {min_fps:.2f}")
print(f" - Max FPS: {max_fps:.2f}")
print(f" - Avg FPS: {avg_fps:.2f}")
print(f" - Avg CPU Usage: {avg_cpu:.2f}%")
print(f" - Elapsed time: {elapsed_time:.2f} seconds")


[INFO] Running face_recognition benchmark for 10 seconds...

[INFO] face_recognition Benchmark Summary:
 - Total frames processed: 68
 - Total faces detected: 44
 - Min FPS: 6.09
 - Max FPS: 7.77
 - Avg FPS: 7.59
 - Avg CPU Usage: 11.89%
 - Elapsed time: 10.06 seconds
